In [8]:
# ✅ Step 1: Ensure Cleanlab-TLM is installed and API key is set
!pip install cleanlab-tlm --quiet

import os
from cleanlab_tlm import TLM

os.environ["CLEANLAB_TLM_API_KEY"] = "0d971ca876c443819414f3b8b8248a25"

# ✅ Step 2: Initialize Cleanlab TLM
tlm = TLM(options={"log": ["explanation"]})

# ✅ Step 3: Define the hallucination-free output generation function
def generate_hallucination_free_output(dish_name, age, disease, recipes_df, profile_df, health_rules, get_similar_dishes, clean_ingredients, get_user_nutrient_goals, llm):
    print(f"🔍 Finding best match for: {dish_name}")
    match = get_similar_dishes(dish_name)[0]
    row = recipes_df[recipes_df["recipe_name"].str.lower() == match.lower()]

    if row.empty:
        return f"❌ Recipe '{match}' not found", None

    raw_ing = row.iloc[0]["ingredients"]
    targets = get_user_nutrient_goals(age)
    disease_lower = disease.strip().lower()

    matched_key = next((key for key in health_rules if key in disease_lower), None)
    if matched_key is None:
        return f"❌ No health rules found for: '{disease}'", None

    avoid = health_rules[matched_key]["avoid"]
    recommend = health_rules[matched_key]["recommend"]

    prompt = build_prompt(
        dish_name=match,
        age=age,
        disease=disease,
        ingredients=raw_ing,
        avoid=avoid,
        recommend=recommend,
        targets=targets
    )

    print("\n📨 Prompt Sent:\n", prompt)
    result = llm.prompt(prompt)
    print("\n🤖 Cleanlab TLM Response:\n", result)
    return result, prompt


# ✅ Step 4: Define variables for the example
dish_name = "pasta"
age = 40
disease = "diabetes, hypertension, heart disease"

# ✅ Step 5: Generate hallucination-free output using Cleanlab TLM
output, prompt = generate_hallucination_free_output(
    dish_name=dish_name,
    age=age,
    disease=disease,
    recipes_df=recipes_df,
    profile_df=profile_df,
    health_rules=health_rules,
    get_similar_dishes=get_similar_dishes,
    clean_ingredients=clean_ingredients,
    get_user_nutrient_goals=get_user_nutrient_goals,
    llm=tlm
)

# ✅ Step 6: Run Cleanlab Trust Scoring
if prompt:
    score_data = tlm.get_trustworthiness_score(prompt=prompt, response=output['response'])

    # ✅ Step 7: Extract and Display Score + Explanation
    trust_score = score_data.get("trustworthiness_score", None)
    explanation = score_data.get("explanation", "No explanation provided.")

    print("🔢 Trust Score:", trust_score)
    print("📝 Explanation:", explanation)
else:
    print(output)

🔍 Finding best match for: pasta

📨 Prompt Sent:
 A 40-year-old patient with diabetes, hypertension, heart disease eats a dish called "pizza pasta".

Ingredients:
- rotini pasta
- lean ground beef
- small onion
- spaghetti sauce
- sliced pepperoni sausage
- shredded mozzarella cheese

Nutrition Goals:
Calories: 2500
Protein: 200g
Carbs: 250g
Fat: 70g
Fiber: 30g

Avoid: sugar, ghee, salt
Recommend from: turmeric, olive oil, cinnamon, spinach, broccoli, kale, bell peppers, mushrooms, zucchini, flaxseeds, chia seeds, whole grain pasta

As a nutrition expert, revise the dish by:
- Removing harmful ingredients
- Adding healthier alternatives
- Explaining how this helps the patient

🤖 Cleanlab TLM Response:
 {'response': 'Certainly! Here\'s a revised version of the "pizza pasta" dish tailored for a 40-year-old patient with diabetes, hypertension, and heart disease, aligned with the nutrition goals and restrictions:\n\n---\n\n### Original Ingredients:\n- Rotini pasta  \n- Lean ground beef  \n-

ValidationError: response type must match prompt type. prompt was provided as str but response is of type <class 'dict'>